### Question 1

Use beautiful soup to crawl wikipedia and create dataframe for Toronto neighborhoods

In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
import pandas as pd

In [27]:
import pickle as pkl

In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [5]:
page = requests.get(url)

In [6]:
soup = BeautifulSoup(page.text, 'html.parser')

In [7]:
table = soup.find("table",{"class":"wikitable sortable"})

In [8]:
table_rows = table.find_all('tr')

In [9]:
table_rows[0].find_all('th')

[<th>Postcode</th>, <th>Borough</th>, <th>Neighbourhood
 </th>]

In [10]:
head = table_rows[0]
columns = head.text.split('\n')[1:4]

In [11]:
l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    if row:
        l.append(row)

In [12]:
df = pd.DataFrame(l, columns=columns)

In [13]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [14]:
# remove the 'Not assigned' for Borough
df = df[df['Borough'] != "Not assigned"]

In [15]:
df['Neighbourhood']

2                     Parkwoods\n
3              Victoria Village\n
4                  Harbourfront\n
5              Lawrence Heights\n
6                Lawrence Manor\n
                  ...            
281    Kingsway Park South West\n
282                   Mimico NW\n
283          The Queensway West\n
284       Royal York South West\n
285              South of Bloor\n
Name: Neighbourhood, Length: 210, dtype: object

In [16]:
# remove '\n' from each string
df = df.replace('\n','', regex=True)

In [17]:
df['Neighbourhood']

2                     Parkwoods
3              Victoria Village
4                  Harbourfront
5              Lawrence Heights
6                Lawrence Manor
                 ...           
281    Kingsway Park South West
282                   Mimico NW
283          The Queensway West
284       Royal York South West
285              South of Bloor
Name: Neighbourhood, Length: 210, dtype: object

In [18]:
# combine 'Neighbourhood' for Postcode, Borough
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

In [19]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [20]:
# verify if Neighbourhood is 'Not assigned'
lo = df[df['Neighbourhood'] == 'Not assigned' ].index.values.astype(int)

# rename 'Neighbourhood' with 'Borough' 
for l in lo:
    df.loc[l]['Neighbourhood'] = df.loc[l]['Borough']

In [21]:
df[df['Neighbourhood'] == 'Not assigned' ]

,Postcode,Borough,Neighbourhood


In [22]:
df.shape

(103, 3)

In [28]:
with open('data.pkl', 'wb') as f:
    pkl.dump(df, f)